In [5]:
# !pip install tensorflow

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import regularizers


# --- Configuration ---
BASE_DATA_DIR = "spectrograms_dataset_chunk_split/"
TRAIN_DIR = f"{BASE_DATA_DIR}/train"
VAL_DIR = f"{BASE_DATA_DIR}/validation"
TEST_DIR = f"{BASE_DATA_DIR}/test"

# Image and Batching parameters (these are important hyperparameters!)
IMG_HEIGHT = 128  # Let's start with 128x128, can be tuned
IMG_WIDTH = 128
BATCH_SIZE = 32

# --- Create TensorFlow Datasets ---
print("--- 📂 Loading Datasets ---")

# Training Dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    label_mode='categorical',  # Important for multi-class classification
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

# Validation Dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR,
    label_mode='categorical',
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

# Test Dataset
test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    label_mode='categorical',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=False # No need to shuffle the test set
)

# Get the class names (raga names) from the directory structure
class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print(f"\nFound {NUM_CLASSES} classes (ragas): {class_names[:5]}...") # Print first 5

# --- Optimize for Performance ---
# Caching and prefetching helps speed up training
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

print("✅ Datasets loaded and optimized successfully!")

--- 📂 Loading Datasets ---
Found 9685 files belonging to 96 classes.
Found 1382 files belonging to 96 classes.
Found 1378 files belonging to 96 classes.

Found 96 classes (ragas): ['Amṛtavarṣiṇi', 'Bahudāri', 'Behāg', 'Bhairavi', 'Bhūṣāvaḷi']...
✅ Datasets loaded and optimized successfully!


In [7]:
# --- Define the CRNN Model ---
print("\n--- 🧠 Building the CRNN Model ---")

# Define the input shape. Spectrograms are usually grayscale, but loaded as 3 channels (RGB) by default.
# We can either convert them to 1 channel or use the 3 channels. Let's stick with 3 for now.
INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)

model = keras.Sequential(name="CRNN_Raga_Classifier")

# === 1. The Convolutional Front-End (Feature Extractor) ===
# We'll normalize the pixel values to be between 0 and 1
model.add(layers.Rescaling(1./255, input_shape=INPUT_SHAPE))

model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

# === 2. The Bridge from CNN to RNN ===
# After the CNN layers, the shape is something like (batch_size, new_height, new_width, channels).
# We need to reshape this to (batch_size, timesteps, features) for the RNN.
# Let's treat the 'new_width' as the timesteps.
# The 'features' will be 'new_height' * 'channels'.
# For an input of (128, 128), after 3 max-pools, the shape is (14, 14, 128).
# So, timesteps=14, features=14*128=1792.

# Get the shape of the output from the last Conv layer to calculate the reshape dims
# This makes the model flexible to input size changes
last_conv_shape = model.output_shape
# last_conv_shape is (None, 14, 14, 128)
timesteps = last_conv_shape[2]  # width
features = last_conv_shape[1] * last_conv_shape[3] # height * channels

model.add(layers.Reshape((timesteps, features)))

# === 3. The Recurrent Back-End (Temporal Modeler) ===
# We use a Bidirectional LSTM to capture context from both past and future timesteps.
model.add(layers.Bidirectional(layers.LSTM(128, return_sequences=True)))
model.add(layers.Dropout(0.5))
model.add(layers.Bidirectional(layers.LSTM(64)))
model.add(layers.Dropout(0.5))


# === 4. The Classifier Head ===
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(NUM_CLASSES, activation='softmax')) # Softmax for multi-class probability

# --- Compile the Model ---
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ Model compiled!")
model.summary()


--- 🧠 Building the CRNN Model ---
✅ Model compiled!


/Users/rahulpeter/miniconda3/envs/carnatic_env/lib/python3.10/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "CRNN_Raga_Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 14, 1792)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 14, 256)        │     1,967,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 14, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 128)            │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 96)             │        12,384 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,254,496 (8.60 MB)

 Trainable params: 2,254,048 (8.60 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
from sklearn.utils import class_weight
import numpy as np

print("\n--- 🚀 Starting Model Training ---")

EPOCHS = 50 # Start with 50 and see how it goes

lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2, 
    patience=3, 
    verbose=1
)
# Callbacks to save the best model and stop early if no improvement
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_raga_model.keras",
        save_best_only=True,
        monitor="val_loss"
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10, # Stop after 10 epochs with no improvement
        verbose=1
    ),
    lr_scheduler
]

train_labels = np.concatenate([y for x, y in train_ds], axis=0)
train_labels_int = np.argmax(train_labels, axis=1)
weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_labels_int),
    y=train_labels_int
)
class_weights = dict(enumerate(weights))

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    # class_weight=class_weights
)

print("\n--- ✅ Training Complete ---")

# --- 📈 Plotting Training History ---
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


# --- 🧪 Evaluating the Best Model on the Test Set ---
print("\n--- Evaluating on Test Data ---")
best_model = keras.models.load_model("best_raga_model.keras")
test_loss, test_acc = best_model.evaluate(test_ds)
print(f"\nTest Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")


--- 🚀 Starting Model Training ---
Epoch 1/50


2025-10-08 00:55:49.882177: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


303/303 ━━━━━━━━━━━━━━━━━━━━ 47s 147ms/step - accuracy: 0.0670 - loss: 4.1043 - val_accuracy: 0.0695 - val_loss: 4.0014 - learning_rate: 0.0010
Epoch 2/50
303/303 ━━━━━━━━━━━━━━━━━━━━ 44s 145ms/step - accuracy: 0.0836 - loss: 3.8956 - val_accuracy: 0.0854 - val_loss: 3.7121 - learning_rate: 0.0010
Epoch 3/50
157/303 ━━━━━━━━━━━━━━━━━━━━ 20s 142ms/step - accuracy: 0.1020 - loss: 3.7769